# Simple Autoencoder

In this notebook we will create a simple autoencoder that just learns a simple system represented by a linear function.

We will use a simple Dense network with Stochastic Gradient Descent optimizer. 

## Importing the Libraries.

We begin by importing our libraries:

In [ ]:
#Simple Autoencoder

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import mean_squared_error
import numpy as np

# For generating our fake data
import random

We now set some hyperparameters, including # of epochs to train, # of datapoints, as well as noise and # of testing data.

Note that we have DATA_NOISE set at approximately 10% of our maximum and minimum data values, and TEST_NOISE set at 30%.

In [ ]:
# Number of epochs to train
EPOCHS = 200

# Number of fake datapoints to create
                                                                                                               
DATA_SIZE = 10000
DATA_NOISE = 0.1
VAL_SIZE = int(0.2 * DATA_SIZE)

# Test Noise level
TEST_NOISE = 0.3

## Creating our Simple Autoencoder

We will now create our autoencoder. We use a fully connected dense network with 3 hidden layers with 32, 16 and 32 hidden nodes. We use ReLu activation in all hidden layers and tanh for the output layer, which allows us to create and learn data from \[-1, 1\]

In [ ]:
# We use the Functional API to create our autoencoder
aeinput = Input(shape = (1, ), name = 'input')
encoder = Dense(units = 4, activation = 'relu')(aeinput)
encoder = Dense(units = 8, activation = 'relu')(encoder)
encoder = Dense(units = 16, activation = 'relu')(encoder) 
decoder = Dense(units = 8, activation = 'relu')(encoder)
decoder = Dense(units = 4, activation = 'relu')(decoder)
aeoutput = Dense(units = 1, activation = 'tanh')(decoder)

ae = Model(aeinput, aeoutput)
ae.compile(loss = 'mean_squared_error', optimizer = 'sgd')


## Generating the Data

Let's now generate the data! We set a fixed random seed (24601.. haha), then create our random datapoints for X between 0 and 1. We then produce our Y data which goes from -1 to 1, and add some synthetic noise.

Finally we call np.array to turn everything into NumPy arrays, which is needed by Keras.

In [ ]:
# We set a fixed seed so that our experiments are reproducible
random.seed(24601)

# This adds white noise of -scale to scale
def noise(scale):
    return (2 * random.uniform(0, 1) - 1) * scale 

def gen_X(data_size, noise_level):
    return [random.uniform(0, 1) + noise(noise_level) for i in range(data_size)]

def gen_WrongX(data_size, noise_level):
    return [random.uniform(0, 1.5) + noise(noise_level) for i in range(data_size)]

# We create our dummy dataset and output, with 1000 numbers
X_in = np.array(gen_X(DATA_SIZE, DATA_NOISE))
X_test = np.array(gen_X(VAL_SIZE, DATA_NOISE))


## Creating Noisy and Wrong Data

We now create our very noisy data, as well as data that is produced using the wrong function ($1.5 \times x$ instead of $2 \times x$), so that we can evaluate how our simple autoencoder behaves.

In [ ]:
# We create a noisy version of Y_out to check the anomaly detection
X_noisy = np.array(gen_X(DATA_SIZE, TEST_NOISE))

# We create an incorrect version of Y_out
X_wrong = np.array(gen_WrongX(DATA_SIZE, DATA_NOISE))


## Training and Evaluation

Finally let's train our autoencoder:

In [ ]:
ae.fit(x = X_in, y = X_in, batch_size = 100, 
epochs = EPOCHS, validation_data = (X_test, X_test))

clean_loss = ae.evaluate(x = X_in, y = X_in)
test_loss = ae.evaluate(x = X_test, y = X_test)
noisy_loss = ae.evaluate(x = X_noisy, y = X_noisy)
wrong_loss = ae.evaluate(x = X_wrong, y = X_wrong)

print("Clean loss = %3.4f, Test loss = %3.4f Noisy loss = %3.4f, Wrong loss = %3.4f" % 
(clean_loss, test_loss, noisy_loss, wrong_loss))

Now let's evaluate the losses for the:

    - Training data
    - Test data
    - Very noisy test data
    - Incorrect data
    
From here we can see how the autoencoder behaves for each type of test:

In [ ]:
clean_loss = ae.evaluate(x = X_in, y = X_in)
test_loss = ae.evaluate(x = X_test, y = X_test)
noisy_loss = ae.evaluate(x = X_noisy, y = X_noisy)
wrong_loss = ae.evaluate(x = X_wrong, y = X_wrong)

print("\n\nClean loss = %3.4f, Test loss = %3.4f Noisy loss = %3.4f, Wrong loss = %3.4f" % 
(clean_loss, test_loss, noisy_loss, wrong_loss))


## Analysis

From the results above, we can see that the test loss is very close to the training loss, but the noisy loss is much larger, and the loss from the incorrect data is a whole magnitude larger, showing that autoencoders are good at detecting abnormal data, making them good for detecting faults in systems.
